# QDC scaling

In [1]:
import xarray as xr
import os
import sys
import glob
import numpy as np
import time
from xclim import sdba
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Import utils
sys.path.append('/home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/')
import utils

# Static metadata dictionaries
from utils import locations, model_dict, vars_1hr, vars_day, cmap_dict

# sys.path.append('/home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/xsdba')
# import xsdba

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.06/lib/python3.11/site-packages/xclim/sdba.py:12: UserWarning: The `xclim.sdba` module has been split into its own package `xsdba`. Users are encouraged to use `xsdba` directly. For the time being, `xclim.sdba` will import `xsdba` to allow for API compatibility. This behaviour may change in the future. For more information, see: https://xsdba.readthedocs.io/en/stable/xclim_migration_guide.html
  warnings.warn(


In [2]:
# ! python -m pip install xsdba
# import xsdba

In [3]:
##### Settings

# Narrow down SSP, RCM and start/end year
_scenario = "ssp370" #"ssp370" ssp126 historical
_rcm = "CCAM-v2203-SN" #"BARPA-R" #CCAM-v2203-SN"
# _gcm = "ACCESS-ESM1-5"
_location = "Melbourne"
_future = "2050"

# Directories
root_dir = "/g/data/eg3/nesp_bff/"
plot_dir = "/g/data/eg3/nesp_bff/plots/qdc_adjustfactor/"

if _rcm == "CCAM-v2203-SN":
    in_dir = f"{root_dir}step1_raw_data_extraction/CSIRO-CCAM/"
    out_dir = f"{root_dir}step2_qdc_scaling/CSIRO-CCAM/"
else:
    in_dir = f"{root_dir}step1_raw_data_extraction/{_rcm}/"
    out_dir = f"{root_dir}step2_qdc_scaling/{_rcm}/"

### QDC daily data

In [ ]:
vars_daily = ["tasmax","tasmin","hussmax","hussmin","psl","sfcWind","sfcWindmax","rsds","rsdsdir"]

print(f"---------- {_rcm} for daily data ----------")

for loc in locations:
    print(f"========================== {loc} =======================")
    ds_ref = xr.open_dataset(f"{root_dir}step1_raw_data_extraction/BARRA-R2/{loc}_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_day_1985-2014.nc").load()

    for _gcm in model_dict[_rcm]["gcms"]:
        print(f"***** {_gcm} *****")

        out_file = (
            f"{out_dir}{loc}_"
            f"{model_dict[_rcm]['grid']}_"
            f"{_gcm}_{_scenario}_"
            f"{model_dict[_rcm]['gcms'][_gcm]['mdl_run']}_"
            f"{model_dict[_rcm]['org']}_"
            f"{_rcm}_{model_dict[_rcm]['gcms'][_gcm]['version']}_"
            f"day_{_future}_QDC-BARRAR2.nc"
        )
        fig_file = plot_dir+out_file.replace(".nc","_AdjFact_facetplot.pdf").split('/')[-1]

        if not os.path.exists(out_file):
            print(f"Processing: {out_file}.....")
            model_historical_file = (
                f"{in_dir}{loc}_"
                f"{model_dict[_rcm]['grid']}_"
                f"{_gcm}_historical_"
                f"{model_dict[_rcm]['gcms'][_gcm]['mdl_run']}_"
                f"{model_dict[_rcm]['org']}_"
                f"{_rcm}_{model_dict[_rcm]['gcms'][_gcm]['version']}_"
                f"day_1985-2014.nc"
            )
            model_future_file = (
                f"{in_dir}{loc}_"
                f"{model_dict[_rcm]['grid']}_"
                f"{_gcm}_{_scenario}_"
                f"{model_dict[_rcm]['gcms'][_gcm]['mdl_run']}_"
                f"{model_dict[_rcm]['org']}_"
                f"{_rcm}_{model_dict[_rcm]['gcms'][_gcm]['version']}_"
                f"day_2035-2064.nc"
            )
            ds_model_historical = xr.open_dataset(model_historical_file).load()
            ds_model_future = xr.open_dataset(model_future_file).load()

            ds_model_historical["lat"] = ds_ref["lat"]
            ds_model_historical["lon"] = ds_ref["lon"]
            ds_model_future["lat"] = ds_ref["lat"]
            ds_model_future["lon"] = ds_ref["lon"]
            ds_model_future_aligned = utils.align_future_to_historical(ds_model_historical, ds_model_future)

            facet_data = []
            var_list = []

            for _var in vars_daily:
                print(f"===== Var: {_var} =====")

                da_ref = ds_ref[_var]
                da_model_historical = ds_model_historical[_var]
                da_model_future = ds_model_future_aligned[_var]

                _kind = '+' if _var in ["tasmax", "tasmin"] else '*'

                QDC = sdba.QuantileDeltaMapping.train(
                    da_model_future,
                    da_model_historical,
                    nquantiles=100,
                    group='time.month',
                    kind=_kind
                )

                da_ref_adjust = QDC.adjust(da_ref, interp='linear').rename(_var)
                var_list.append(da_ref_adjust.to_dataset())

                if not os.path.exists(fig_file):
                    def plot_mean(ax, var=_var, da_ref=da_ref, da_model_historical=da_model_historical, da_model_future=da_model_future, da_ref_adjust=da_ref_adjust):
                        da_ref.groupby("time.dayofyear").mean().plot(ax=ax, label="Reference historical (BARRA-R2, 2000)")
                        da_model_historical.groupby("time.dayofyear").mean().plot(ax=ax, label=f"{_rcm}-{_gcm} historical (2000)")
                        da_model_future.groupby("time.dayofyear").mean().plot(ax=ax, label=f"{_rcm}-{_gcm} {_scenario} - future ({_future})")
                        da_ref_adjust.groupby("time.dayofyear").mean().plot(ax=ax, label="Reference - adjusted - future", linestyle="--")
                        ax.set_title(f"{loc}, {var} - Mean Daily")
                        ax.legend()
    
                    facet_data.append(plot_mean)
    
                    def plot_qdc_af(ax, var=_var, qdc=QDC, kind=_kind, cmap_default=cmap_dict.get(_var, "viridis")):
                        af = qdc.ds.af
                        vmin = float(af.min())
                        vmax = float(af.max())
    
                        if vmin < 0 and vmax > 0:
                            cmap = "RdBu_r"
                            center = 0.0
                        else:
                            cmap = cmap_default
                            center = None
    
                        af.transpose("month", "quantiles").plot.pcolormesh(
                            ax=ax, cmap=cmap, center=center,
                            x="quantiles", y="month",
                            cbar_kwargs={"label": "Adjustment Factor"})
                        ax.set_title(f"{var} - Adjustment Factors ({kind})")
    
                    facet_data.append(plot_qdc_af)
    
                    def plot_hist(ax, var=_var, da_ref=da_ref, da_model_historical=da_model_historical, da_model_future=da_model_future, da_ref_adjust=da_ref_adjust):
                        sns.histplot(da_ref.values.flatten(), ax=ax, label="Obs (BARRA-R2)", kde=True, stat="density", bins=50, color="blue")
                        sns.histplot(da_model_historical.values.flatten(), ax=ax, label=f"Hist {_rcm}-{_gcm} {_scenario}", kde=True, stat="density", bins=50, color="green")
                        sns.histplot(da_model_future.values.flatten(), ax=ax, label=f"2050 {_rcm}-{_gcm} {_scenario}", kde=True, stat="density", bins=50, color="red")
                        sns.histplot(da_ref_adjust.values.flatten(), ax=ax, label="Adjusted", kde=True, stat="density", bins=50, color="purple")
                        ax.set_title(f"{loc}, {var} - Distribution")
                        ax.legend()
    
                    facet_data.append(plot_hist)

            da_var = xr.merge(var_list)
            print(da_var)
            da_var.to_netcdf(out_file)

            if not os.path.exists(fig_file):
                fig, axes = plt.subplots(9, 3, figsize=(30, 45))
                axes = axes.flatten()
    
                for ax, plot_func in zip(axes, facet_data):
                    plot_func(ax)
    
                plt.tight_layout()
                fig.savefig(fig_file, bbox_inches='tight')
                # plt.show()
                plt.close()

        else:
            print(f'File for {loc} exists in output directory.')


## QDC hourly data

In [ ]:
vars_1hr = ['tas','hurs','huss','sfcWind','psl','uas','vas','clt','rsds','rsdsdir']

print(f"---------- {_rcm} for hourly data ----------")
issue_list = []

for loc in locations:
    print(f"========================== {loc} =======================")
    ds_ref = xr.open_dataset(f"{root_dir}step1_raw_data_extraction/BARRA-R2/{loc}_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_1985-2014.nc").load()

    for _gcm in model_dict[_rcm]["gcms"]:
        print(f"***** {_gcm} *****")

        out_file = (
            f"{out_dir}{loc}_"
            f"{model_dict[_rcm]['grid']}_"
            f"{_gcm}_{_scenario}_"
            f"{model_dict[_rcm]['gcms'][_gcm]['mdl_run']}_"
            f"{model_dict[_rcm]['org']}_"
            f"{_rcm}_{model_dict[_rcm]['gcms'][_gcm]['version']}_"
            f"1hr_{_future}_QDC-BARRAR2.nc"
        )
        # fig_file = plot_dir+out_file.replace(".nc","_AdjFact_facetplot.pdf").split('/')[-1]
        try:
            if not os.path.exists(out_file):
                print(f"Processing: {out_file}.....")
                model_historical_file = (
                    f"{in_dir}{loc}_"
                    f"{model_dict[_rcm]['grid']}_"
                    f"{_gcm}_historical_"
                    f"{model_dict[_rcm]['gcms'][_gcm]['mdl_run']}_"
                    f"{model_dict[_rcm]['org']}_"
                    f"{_rcm}_{model_dict[_rcm]['gcms'][_gcm]['version']}_"
                    f"1hr_1985-2014.nc"
                )
                model_future_file = (
                    f"{in_dir}{loc}_"
                    f"{model_dict[_rcm]['grid']}_"
                    f"{_gcm}_{_scenario}_"
                    f"{model_dict[_rcm]['gcms'][_gcm]['mdl_run']}_"
                    f"{model_dict[_rcm]['org']}_"
                    f"{_rcm}_{model_dict[_rcm]['gcms'][_gcm]['version']}_"
                    f"1hr_2035-2064.nc"
                )
                ds_model_historical_1hr = xr.open_dataset(model_historical_file).load()
                ds_model_future_1hr = xr.open_dataset(model_future_file).load()
    
                ds_model_historical_1hr["lat"] = ds_ref["lat"]
                ds_model_historical_1hr["lon"] = ds_ref["lon"]
                ds_model_future_1hr["lat"] = ds_ref["lat"]
                ds_model_future_1hr["lon"] = ds_ref["lon"]
                ds_model_future_1hr_aligned = utils.align_future_to_historical(ds_model_historical_1hr, ds_model_future_1hr)
    
                facet_data = []
                var_list_1hr = []
    
                for _var in vars_1hr:
                    print(f"===== Var: {_var} =====")
    
                    da_obs_1hr = ds_ref[_var]
                    da_model_hist_1hr = ds_model_historical_1hr[_var]
                    da_model_fut_1hr = ds_model_future_1hr_aligned[_var]
    
                    _kind = '+' if _var in ["tas","hurs", "clt", "uas", "vas"] else '*'
                                    
                    da_adjusted_1hr, QDC_dict_1hr, adjusted_slices_1hr = utils.apply_hourly_qdc_sliding_window(
                        da_obs=da_obs_1hr,
                        da_model_historical=da_model_hist_1hr,
                        da_model_future=da_model_fut_1hr,
                        var=_var,
                        kind=_kind,
                        window=1
                    )
                    
                    var_list_1hr.append(da_adjusted_1hr.rename(_var).to_dataset())
    
                    fig_file = plot_dir+(out_file.replace(".nc",f"_AdjFact_facetplot_{_var}.pdf").split('/')[-1])
                    if _var in ["rsds","rsdsdir"]:
                        if not os.path.exists(fig_file):
                            plot_diagnostics = utils.plot_qdc_hourly_diagnostics(da_obs_1hr,da_model_hist_1hr,
                                                                                 da_model_fut_1hr,QDC_dict_1hr,
                                                                                 adjusted_slices_1hr,_var,
                                                                                 f"{_rcm}-{_gcm}",loc)
                            plt.tight_layout()
                            plt.savefig(fig_file, bbox_inches='tight')
                            # plt.show()
                            plt.close()
                        
                da_var_1hr = xr.merge(var_list_1hr)
                print(da_var_1hr)
                da_var_1hr.to_netcdf(out_file)
            else:
                print(f'File for {loc} exists in output directory.')
                
        except Exception as e:
            issue_list.append(f"Issue with {_var} in {_gcm} for {loc}. File not created: {out_file}. Error:{e}")
            print(f"Issue with {_var} in {_gcm}. Not creating {out_file}. Skipping to next. Error:{e}")


In [ ]:
for item in issue_list:
    print("Summary of issues and files not created because of it: \n",item,"\n")

In [ ]:
# vars_1hr = ['rsds','rsdsdir']

# in_dir = "/g/data/eg3/nesp_bff/step1_raw_data_extraction/BARPA-R/"
# root_dir = "/g/data/eg3/nesp_bff/"

# ds_model_future_1hr = xr.open_dataset(f"{in_dir}Cairns_AUS-15_CMCC-ESM2_ssp370_r1i1p1f1_BOM_BARPA-R_v1-r1_1hr_2035-2064.nc").load()
# ds_model_historical_1hr = xr.open_dataset(f"{in_dir}Cairns_AUS-15_CMCC-ESM2_historical_r1i1p1f1_BOM_BARPA-R_v1-r1_1hr_1985-2014.nc").load()
# ds_ref_1hr = xr.open_dataset(f"{root_dir}step1_raw_data_extraction/BARRA-R2/Cairns_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_1985-2014.nc").load()

# ds_model_historical_1hr["lat"] = ds_ref_1hr["lat"]
# ds_model_historical_1hr["lon"] = ds_ref_1hr["lon"]
# ds_model_future_1hr["lat"] = ds_ref_1hr["lat"]
# ds_model_future_1hr["lon"] = ds_ref_1hr["lon"]
# ds_model_future_1hr_aligned = utils.align_future_to_historical(ds_model_historical_1hr, ds_model_future_1hr)

# facet_data = []
# var_list_1hr = []

# for _var in vars_1hr:
#     print(f"===== Var: {_var} =====")
    
#     da_obs_1hr = ds_ref_1hr[_var]
#     da_model_hist_1hr = ds_model_historical_1hr[_var]
#     da_model_fut_1hr = ds_model_future_1hr_aligned[_var]
    
#     print(da_obs_1hr)
#     print(da_model_hist_1hr)
#     print(da_model_fut_1hr)
    
#     _kind = '+' if _var in ["tas","hurs", "clt", "uas", "vas"] else '*'
                                    
#     da_adjusted_1hr, QDC_dict_1hr, adjusted_slices_1hr = utils.apply_hourly_qdc_sliding_window(
#         da_obs=da_obs_1hr,
#         da_model_historical=da_model_hist_1hr,
#         da_model_future=da_model_fut_1hr,
#         var=_var,
#         kind=_kind,
#         window=1
#         )
                    
#     var_list_1hr.append(da_adjusted_1hr.rename(_var).to_dataset())




In [5]:
vars_1hr = ['rsds','rsdsdir']

in_dir = "/g/data/eg3/nesp_bff/step1_raw_data_extraction/CSIRO-CCAM/"
root_dir = "/g/data/eg3/nesp_bff/"

ds_model_future_1hr = xr.open_dataset(f"{in_dir}Thredbo_AUS-10i_ACCESS-ESM1-5_ssp370_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_2035-2064.nc").load()
ds_model_historical_1hr = xr.open_dataset(f"{in_dir}Thredbo_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_1985-2014").load()
ds_ref_1hr = xr.open_dataset(f"{root_dir}step1_raw_data_extraction/BARRA-R2/Thredbo_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_day_1985-2014.nc").load()

ds_model_historical_1hr["lat"] = ds_ref_1hr["lat"]
ds_model_historical_1hr["lon"] = ds_ref_1hr["lon"]
ds_model_future_1hr["lat"] = ds_ref_1hr["lat"]
ds_model_future_1hr["lon"] = ds_ref_1hr["lon"]
ds_model_future_1hr_aligned = utils.align_future_to_historical(ds_model_historical_1hr, ds_model_future_1hr)

facet_data = []
var_list_1hr = []

for _var in vars_1hr:
    print(f"===== Var: {_var} =====")
    
    da_obs_1hr = ds_ref_1hr[_var]
    da_model_hist_1hr = ds_model_historical_1hr[_var]
    da_model_fut_1hr = ds_model_future_1hr_aligned[_var]
    
    print(da_obs_1hr)
    print(da_model_hist_1hr)
    print(da_model_fut_1hr)
    
    _kind = '+' if _var in ["tasmax", "tasmin"] else '*'

    QDC = sdba.QuantileDeltaMapping.train(
        da_model_future,
        da_model_historical,
        nquantiles=100,
        group='time.month',
        kind=_kind
    )

    da_ref_adjust = QDC.adjust(da_ref, interp='linear').rename(_var)

    def plot_mean(ax, var=_var, da_ref=da_ref, da_model_historical=da_model_historical, da_model_future=da_model_future, da_ref_adjust=da_ref_adjust):
        da_ref.groupby("time.dayofyear").mean().plot(ax=ax, label="Reference historical (BARRA-R2, 2000)")
        da_model_historical.groupby("time.dayofyear").mean().plot(ax=ax, label=f"{_rcm}-{_gcm} historical (2000)")
        da_model_future.groupby("time.dayofyear").mean().plot(ax=ax, label=f"{_rcm}-{_gcm} {_scenario} - future ({_future})")
        da_ref_adjust.groupby("time.dayofyear").mean().plot(ax=ax, label="Reference - adjusted - future", linestyle="--")
        ax.set_title(f"{loc}, {var} - Mean Daily")
        ax.legend()
    
    facet_data.append(plot_mean)
    
    def plot_qdc_af(ax, var=_var, qdc=QDC, kind=_kind, cmap_default=cmap_dict.get(_var, "viridis")):
        af = qdc.ds.af
        vmin = float(af.min())
        vmax = float(af.max())
    
        if vmin < 0 and vmax > 0:
            cmap = "RdBu_r"
            center = 0.0
        else:
            cmap = cmap_default
            center = None
    
        af.transpose("month", "quantiles").plot.pcolormesh(
            ax=ax, cmap=cmap, center=center,
            x="quantiles", y="month",
            cbar_kwargs={"label": "Adjustment Factor"})
        ax.set_title(f"{var} - Adjustment Factors ({kind})")
    
    facet_data.append(plot_qdc_af)
    
    def plot_hist(ax, var=_var, da_ref=da_ref, da_model_historical=da_model_historical, da_model_future=da_model_future, da_ref_adjust=da_ref_adjust):
        sns.histplot(da_ref.values.flatten(), ax=ax, label="Obs (BARRA-R2)", kde=True, stat="density", bins=50, color="blue")
        sns.histplot(da_model_historical.values.flatten(), ax=ax, label=f"Hist {_rcm}-{_gcm} {_scenario}", kde=True, stat="density", bins=50, color="green")
        sns.histplot(da_model_future.values.flatten(), ax=ax, label=f"2050 {_rcm}-{_gcm} {_scenario}", kde=True, stat="density", bins=50, color="red")
        sns.histplot(da_ref_adjust.values.flatten(), ax=ax, label="Adjusted", kde=True, stat="density", bins=50, color="purple")
        ax.set_title(f"{loc}, {var} - Distribution")
        ax.legend()
    
    facet_data.append(plot_hist)





ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'argo', 'cfgrib', 'cfradial1', 'datamet', 'era5', 'funwave', 'furuno', 'gamic', 'gini', 'gmt', 'hpl', 'iris', 'json', 'kerchunk', 'metek', 'ncswan', 'ndbc', 'ndbc_ascii', 'netcdf', 'nexradlevel2', 'octopus', 'odim', 'pydap', 'radolan', 'rainbow', 'rasterio', 'spotter', 'swan', 'triaxys', 'wavespectra', 'ww3', 'ww3_station', 'wwm', 'xwaves', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html